# Sitemap 
In a first step we scrape the sitemap to get the urls for all camping sites.

In [1]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import json

def scrape_sitemap(url):
    # Fetch the sitemap XML file
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML content
        root = ET.fromstring(response.content)
        
        # Create a list to store the URLs
        urls = []
        
        # Extract the URLs from the XML
        for element in root.iter():
            if 'url' in element.tag:
                for loc in element.iter():
                    if 'loc' in loc.tag:
                        urls.append(loc.text)
        
        return urls
    
    else:
        # If the request failed, print an error message
        print(f"Failed to fetch sitemap: {response.status_code}")
        return None

# URL of the sitemap
sitemap_url = 'https://www.gocamping.ch/sitemap_total.xml'

# Scrape the sitemap and store the URLs in a list
sitemap_raw = scrape_sitemap(sitemap_url)

In [2]:
len(sitemap_raw)

878

There are too many urls. Duplicates found. This code removes all duplicates.

In [3]:
sitemap_data = []
for site in sitemap_raw:
    if site in sitemap_data:
        continue
    else:
        sitemap_data.append(site)

In [4]:
len(sitemap_data)

439

In [5]:
import csv

# Write sitemap to file

# Open the file in write mode
with open('sitemap.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # Write multiple rows
    for site in sitemap_data:
        writer.writerow([site])


In [2]:
import csv

# Import sitemap from file

sitemap_data = []

with open('sitemap.csv') as sitemap:
    csv_reader = csv.reader(sitemap, delimiter=',')
    for row in csv_reader:
        sitemap_data.append(row[0])


# Scraping
Scraping of the data

In [7]:
sample_url = ['https://www.gocamping.ch/hotels/280/Camping-La-Grande-Ecluse.html']

In [4]:
# Function for retrieving ldjson for camping site
import html

def extract_ldjson(soup):
    ldjson = soup.find('script', attrs={'type': 'application/ld+json'}).string
    ldjson = html.unescape(ldjson)
    data = json.loads(ldjson)
    return data

In [5]:
# Function for retrieving price data for camping site
import re
def extract_price(soup):
    # Find the divs with the price categories in the div with the id "ausstattung_section" and 
    ausstattung_section = soup.find("div", id="ausstattung_section")
    price_categories = ausstattung_section.find_all("div", class_="roomslistingsingle")

    # Find the divs with the class "price_details" and "longdescription" within the "ausstattung_section" div
    price_listings = {}
    
    #print(longdescription_divs)
    for category in price_categories:
        # print(category)
        title = category.find('h4')
        description = category.find("div", class_='content')
        price_text = category.find("div", class_='price_details')
        price = re.findall(r'\d+\.\d+', price_text.string)
        price_listings[title.string] = {'description': description.string, 'price': price[0]}
        
    return price_listings

In [25]:
# Function for retrieving tourist_tax data for camping site
import pandas as pd
def extract_tax(soup):
    # Find tax table on website
    tax_section = soup.find('div', {'id': 'hotel-payment_section'})
    tax_table = tax_section.findChildren('table')

    # Convert html to df. We must remove apostrophs in numbers like "CHF 1'234"
    df = pd.read_html(str(tax_table).replace("'", ""))[0]

    # Clean up table
    df.set_axis(['tax_category', 'off_season', 'main_season'], axis=1, inplace=True) # Add column headers
    df.set_axis(df['tax_category'].tolist(), axis='rows', inplace=True) # Add row headers
    df.drop(['tax_category'], axis='columns', inplace=True) # Remove column that contains headers
    df.drop(df.tail(1).index, axis='rows', inplace=True) # Remove last row with accepted payment methods
    pos = [0,1] # Define irrelevant rows on top that need to be removed
    df.drop(df.index[pos], axis='index', inplace=True) # Remove irrelevant top rows
    df['off_season'] = df['off_season'].str.replace('CHF', '').astype('float')
    df['main_season'] = df['main_season'].str.replace('CHF', '').astype('float')

    return df.to_dict()


In [18]:
# Function for extracting all data from one url
def extract_data(url):
    # Fetch the website
    response = requests.get(url)
    data = ''
    extract_error = ""
    
    # Check if the request was successful
    if response.status_code == 200:
        
        # The gocamping site can contain invalid control characters like carriage return
        # or alt-space. We need to replace these before we invoke BeautifulSoup
        htmlString = response.content.decode('utf-8').replace("\r", "").replace("\n","").replace("\xa0", "")
        
        # Create a BeautifulSoup object from the response content
        soup = BeautifulSoup(htmlString, "html.parser")
        
        # Extract all data from website
        try:
            desc = extract_ldjson(soup)
            price = extract_price(soup)
            tax = extract_tax(soup)
            
            data = {'description': extract_ldjson(soup), 'price': extract_price(soup), 'tax': extract_tax(soup)}
        except Exception as e:
            print('extraction failed for', url, str(e))
            extract_error = e
    
    else:
        # If the request failed, print an error message
        print(f"Failed to access site: {response.status_code}")
        extract_error = response.status_code
    
    return data, extract_error

In [24]:
import requests
from bs4 import BeautifulSoup
import time
import json
  
# Read already scraped camping sites from JSON file
try:
    f = open('camping_sites.json')
    sites = json.load(f) # returns JSON object as a dictionary
    f.close()
except FileNotFoundError:
    sites = {}
    

data = {}
errors = []

# Scrape the data for all sites not already listed in the file
for url in sitemap_data:
    if url not in sites or sites[url] == "": # Checking if data is already in file or file listing is empty
        site, extract_error = extract_data(url)
        if extract_error == "":
            data[url] = site
            print(url, "harvested successfully")
        else:
            errors.append([url, extract_error])
        time.sleep(10)
    else:
        print(url, 'skipped because it was already scraped')
        data[url] = sites[url]

# Writing json_data to a file
json_object = json.dumps(data, indent=4, ensure_ascii=False)
with open("camping_sites.json", "w") as outfile:
    outfile.write(json_object)

extraction failed for https://www.gocamping.ch/hotels/203/Oliver-Schibli.html 'NoneType' object has no attribute 'find_all'
[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01.-30.06 / 01.09 - 31.12.</b></td><td><b>01.07.-31.08.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 10.00</td><td>CHF 13.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 5.00</td><td>CHF 6.50</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 5.00</td><td>CHF 6.50</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.90</td><td>CHF 0.90</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkei

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01 - 31.12</b></td><td><b>01.01 - 31.12</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 9.90</td><td>CHF 9.90</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 4.30</td><td>CHF 4.30</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 4.30</td><td>CHF 4.30</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 3.50</td><td>CHF 3.50</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 3.30</td><td>CHF 3.30</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 4.30</td><td>CHF 4.30</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_cash.png" style="height: 26px; margin-right: 5px;"/></td></tr> </table>]
 

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.04 - 24.10.</b></td><td><b>01.04 - 24.10.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 14.00</td><td>CHF 14.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 6.00</td><td>CHF 6.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 6.00</td><td>CHF 6.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 5.00</td><td>CHF 5.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 2.30</td><td>CHF 2.30</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 5.00</td><td>CHF 5.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_visa.png" style="height: 26px; margin-right: 5px;"><img src="templates

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>25.03 - 01.10</b></td><td><b>25.03 - 01.10</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 7.50</td><td>CHF 7.50</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 4.50</td><td>CHF 4.50</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 1.00</td><td>CHF 1.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 3.00</td><td>CHF 3.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_cash.png" style="height: 26px; margin-right: 5px;"/></td></tr> </table>]
 

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>17.5. - 30.6. / 1.9. - 15.10.</b></td><td><b>1.7. - 31.8</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 12.00</td><td>CHF 13.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 10.00</td><td>CHF 11.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 8.00</td><td>CHF 9.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 4.00</td><td>CHF 4.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_visa.png" style="height: 26px; margin-right: 5px;"><img 

...

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01 - 31.12.</b></td><td><b>01.01 - 31.12.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 5.00</td><td>CHF 5.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_amex.png" style="height: 26px; margin-right: 5px; "><img src="templates/

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01 - 31.12.</b></td><td><b>01.01 - 31.12.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 7.00</td><td>CHF 7.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 2.00</td><td>CHF 2.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 2.00</td><td>CHF 2.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 4.00</td><td>CHF 4.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_cash.png" style="height: 26px; margin-right: 5px;"/></td></tr> </table>]

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.05 - 31.10.</b></td><td><b>01.05 - 31.10.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_visa.png" style="height: 26px; margin-right: 5px;"><img src="templates/c

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01 - 31.12.</b></td><td><b>01.01 - 31.12.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 4.00</td><td>CHF 4.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_visa.png" style="height: 26px; margin-right: 5px;"><img src="templates/c

[<table width="100%"> <tr class="pricetable_date_season"><td></td><td><b>Nebensaison</b></td><td><b>Hauptsaison</b></td></tr> <tr class="pricetable_date_season"><td><b>Öffnungszeiten</b></td><td><b>01.01. - 31.12.</b></td><td><b>01.01. - 31.12.</b></td></tr> <tr class="odd pricetable"><td><b>Preis Erwachsene</b></td><td>CHF 10.00</td><td>CHF 10.00</td></tr> <tr class="pricetable"><td><b>Preis Jugendliche</b></td><td>CHF 5.00</td><td>CHF 5.00</td></tr> <tr class="odd pricetable"><td><b>Preis Kind</b></td><td>CHF 5.00</td><td>CHF 5.00</td></tr> <tr class="pricetable"><td><b>Preis Hund</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Kurtaxe</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="pricetable"><td><b>Strom</b></td><td>CHF 0.00</td><td>CHF 0.00</td></tr> <tr class="odd pricetable"><td><b>Zahlungsmöglichkeiten</b></td><td colspan="2"> <img src="templates/custom/images/icon_cash.png" style="height: 26px; margin-right: 5px;"/></td></tr> </tab

In [26]:
print('success:', len(data), 'failed:', len(errors))

success: 403 failed: 36
